In [10]:
from fastmri.data import transforms as T
import fastmri
import h5py
import numpy as np
from matplotlib import pyplot as plt

from utils_tf import virtual_coil_reconstruction, show_coils
from testgpt_file import TFV

In [11]:
file_path = "/volatile/FastMRI/brain_multicoil_train/multicoil_train/file_brain_AXT1POST_201_6002780.h5"
hf = h5py.File(file_path)

In [12]:
volume_kspace = hf['kspace'][()]
print(volume_kspace.dtype)
print(volume_kspace.shape)
# print(volume_kspace)

complex64
(16, 16, 640, 320)


In [13]:
combined_image = TFV(volume_kspace)
print(combined_image.shape)

2023-12-15 16:28:54.188286: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.00MiB (rounded to 419430400)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-15 16:28:54.188339: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-12-15 16:28:54.188356: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 6, Chunks in use: 5. 1.5KiB allocated for chunks. 1.2KiB in use in bin. 21B client-requested in use in bin.
2023-12-15 16:28:54.188372: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-12-15 16:28:54.188380: I external/lo

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [4]:
slice_kspace = Nvolume_kspace[Nvolume_kspace.shape[0]-1] # Choosing the lastest slice of this volume
print(slice_kspace.shape)

slice_kspace2 = T.to_tensor(slice_kspace)      # Convert from numpy array to pytorch tensor
print(slice_kspace2.shape)
slice_image = fastmri.ifft2c(slice_kspace2)           # Apply Inverse Fourier Transform to get the complex image
print(slice_image.shape)
slice_image_abs = fastmri.complex_abs(slice_image)   # Compute absolute value to get a real image
print(slice_image_abs.shape)

(16, 640, 320, 1)
torch.Size([16, 640, 320, 1, 2])
torch.Size([16, 640, 320, 1, 2])
torch.Size([16, 640, 320, 1])


In [5]:
# new_image_singlecoil = virtual_coil_reconstruction(Nvolume_kspace)
new_image_singlecoil = virtual_coil_reconstruction(slice_image_abs)


2023-12-15 16:12:22.645368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 149 MB memory:  -> device: 0, name: Quadro M4000, pci bus id: 0000:03:00.0, compute capability: 5.2
2023-12-15 16:12:34.858066: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.81GiB (rounded to 8388608000)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-15 16:12:34.858124: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-12-15 16:12:34.858148: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 6, Chunks in use: 5. 1.5KiB allocated for chunks. 1.2KiB in use in bin. 21B client-requested in use in bin.
2023-12-15 

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

In [ ]:
new_image_singlecoil = virtual_coil_reconstruction(slice_image)
print(new_image_singlecoil.shape)
plt.imshow(np.abs(new_image_singlecoil.numpy()), cmap='gray')

2023-12-15 15:22:18.600541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 187 MB memory:  -> device: 0, name: Quadro M4000, pci bus id: 0000:03:00.0, compute capability: 5.2
2023-12-15 15:22:29.586007: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 50.00MiB (rounded to 52428800)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-15 15:22:29.586127: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-12-15 15:22:29.586145: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 2, Chunks in use: 2. 512B allocated for chunks. 512B in use in bin. 12B client-requested in use in bin.
2023-12-15 15:22

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 